In [1]:
from selenium import webdriver
import urllib
import os
import requests
import io
from PIL import *
from PIL import Image
import hashlib
import time
from selenium.webdriver.common.by import By

In [2]:
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# s=Service(ChromeDriverManager().install())

In [3]:
DRIVER_PATH = r'C:\Users\tapen\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe'

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
ser = Service(DRIVER_PATH)
op = webdriver.ChromeOptions()

In [21]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls


In [22]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


In [23]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=50):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(service=ser, options=op) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [24]:
serch_term = 'person wearing face mask'
search_and_download(search_term=serch_term, driver_path=DRIVER_PATH)

<ipython-input-21-6fae6806c786>:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


<ipython-input-21-6fae6806c786>:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 50 image links, done!
SUCCESS - saved https://m.economictimes.com/thumb/msid-77782370,width-1200,height-900,resizemode-4,imgsize-319096/masks-may-protect-the-people-wearing-them-by-reducing-the-amount-of-droplets-from-others-that-might-make-contact-with-them-.jpg - as ./images\person_wearing_face_mask\0fb6b36af4.jpg
SUCCESS - saved https://image.cnbcfm.com/api/v1/image/106467352-1585602933667virus-medical-flu-mask-health-protection-woman-young-outdoor-sick-pollution-protective-danger-face_t20_o07dbe.jpg?v=1585602987 - as ./images\person_wearing_face_mask\2d7d0ca293.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQkciCDI2_CBzKLEqKnqTjOW5xKSzYg4OHmbg&usqp=CAU - as ./images\person_wearing_face_mask\92e716813a.jpg
SUCCESS - saved https://i.insider.com/5e583363fee23d3fd10eac33?width=1136&format=jpeg - as ./images\person_wearing_face_mask\790738e811.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTe75ZSN1AldHBqdLGduThuq0fD9d3qd2pCRg&

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSIjOqxqgH4MKgqgspj4n8EAsrNm17S3f0QIw&usqp=CAU - as ./images\person_wearing_face_mask\6034616e12.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSTfoGWa65BWuerNPb3T9zi60VvAYa9FoYxIw&usqp=CAU - as ./images\person_wearing_face_mask\a14d271b1f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRvAET3hltcjC1gD5QvuVamrE6fMRYg4YVkvQ&usqp=CAU - as ./images\person_wearing_face_mask\960d83dcc2.jpg
SUCCESS - saved https://vitalrecord.tamhsc.edu/wp-content/uploads/2020/04/woman-wearing-face-mask-3902882-1100x625.jpg - as ./images\person_wearing_face_mask\82a8b5c26c.jpg
SUCCESS - saved https://www.onemedical.com/media/images/woman-wearing-mask.original.jpg - as ./images\person_wearing_face_mask\2c32d3b869.jpg
